In [70]:
import json
import sys
import datetime
import re
import os
import time
from birdy.twitter import UserClient, BirdyException 
import numpy as np
from time import sleep
import pandas as pd
import pickle

key = [
    ["", "", "", ""]
]

client = UserClient(key[0][0], key[0][1], key[0][2], key[0][3])

In [49]:
# 0. id_str				: ID of the user
# 1. screen_name		: Screen name of the user (@screen_name)
# 2. followers_count	: # Followers
# 3. friends_count		: # Following
# 4. favourites_count	: # Likes
# 5. listed_count		: Total number of list subscription and membership (?)
# 6. statuses_count		: # Tweets
# 7. verified			: True or False 
# 8. protected			: True or False / if true can't crawl the account
# 9. created_at			: Creation time of the account / (2009-10-30 12:11:39)

baseUsers = []
baseLists = []
usernames = []

f = open("SimilarUsers.txt", 'r', encoding='utf-8')

# read base users
baseUsers = f.readline().split(',')
print(baseUsers)

# read lists
listNumber = int(f.readline())
print(listNumber)
for i in range(listNumber):
    baseLists.append(f.readline().split(','))

# read similar users
for l in f.readlines():
    if l.split(',')[8] == "False":
        usernames.append(l.split(',')[1])

f.close()

print(len(usernames))
print(usernames)



['karpathy', 'AndrewYNg', 'drfeifei', 'AlecRad', 'KirkDBorne', 'hmason', 'hadleywickham\n']
4
567
['NateSilver538', 'googleresearch', 'googleanalytics', 'dez_blanchfield', 'TwitterData', 'HansRosling', 'MongoDB', 'ValaAfshar', 'avinash', 'analyticbridge', 'IBMBigData', 'dr_morton', 'DARPA', 'KirkDBorne', 'simonlporter', 'hortonworks', 'AndrewYNg', 'GreatAnalytics', 'Informatica', 'IBMcloud', 'IBMAnalytics', 'tableau', 'cloudera', 'MaxCRoser', 'jeremywaite', 'hmason', 'DataScienceCtrl', 'AKQA', 'BernardMarr', 'Ronald_vanLoon', 'gruset', 'dhawaldamania', 'kdnuggets', 'mitsmr', 'MSPowerBI', 'kaggle', 'DataSciFact', 'flowingdata', 'JDavidMorris', 'mikejulietbravo', 'plotlygraphs', 'drfeifei', 'BigDataGal', 'EvanSinar', 'SebastianThrun', 'karpathy', 'mgualtieri', 'Mark_Carey', 'Doug_Laney', 'OttLegalRebels', 'EXAGolo', 'ylecun', 'ipfconline1', 'simplystats', 'Rbloggers', 'mapr', '7wData', 'BigDataBlogs', 'MrDataScience', 'hadleywickham', 'DimensionData', 'pivotal', 'BoozAllen', 'rstudio', '

In [61]:
userTweets = []

keyInd = 1
client = UserClient(key[keyInd][0], key[keyInd][1], key[keyInd][2], key[keyInd][3])

for user in usernames:
    filename = "tweets/" + user + ".txt"
    f = open(filename, 'w', encoding='utf-8')

    print(user)
    protec = False
    twe = []
    
    while(True):
        try:
            response = client.api.statuses.user_timeline.get(screen_name=user, count=200)
            break
        except Exception as err:
            print(err.status_code)
            print(err)
            if err.status_code == 429:
                sleep(60)
                keyInd = (keyInd + 1)%len(key)
            elif err.status_code == 404:
                protec = True
                break
            else:
                sleep(15)
            
            client = UserClient(key[keyInd][0], key[keyInd][1], key[keyInd][2], key[keyInd][3])
            
    if protec or len(response.data) == 0:
        userSubs.append([])
        print('protected!')
        continue
        
    for d in response.data:
        twe.append(d['text'])
        f.write(d['text'])
        f.write(" ")
        
    maxID = response.data[-1]['id']
    
    while(True):
        while(True):
            try:
                response = client.api.statuses.user_timeline.get(screen_name=user, count=200, max_id = maxID-1)
                break
            except Exception as err:
                print(err.status_code)
                print(err)
                if err.status_code == 429:
                    sleep(60)
                    keyInd = (keyInd + 1)%len(key)
                else:
                    sleep(15)
                client = UserClient(key[keyInd][0], key[keyInd][1], key[keyInd][2], key[keyInd][3])
        
        if len(response.data) == 0:
            break
        
        for d in response.data:
            twe.append(d['text'])
            f.write(d['text'])
            f.write(" ")
            
        maxID = response.data[-1]['id']
            
    userTweets.append(twe)
    f.close()

FrankPasquale
BaiduResearch
